# In this notebook I will go through an approach to address class imbalance through oversampling the under represented data

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import math 
from matplotlib.figure import Figure
pd.set_option('display.max_colwidth', None)
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
file=pd.read_excel('Bank_Personal_Loan_Modelling.xlsx', sheet_name='Data')

In [3]:
details=pd.read_excel('Bank_Personal_Loan_Modelling.xlsx')

In [4]:
details

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Data Description:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,ID,Customer ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Age,Customer's age in completed years,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,Experience,#years of professional experience,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Income,Annual income of the customer ($000),NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
y=file['Personal Loan']

In [6]:
X=file.drop('Personal Loan', axis='columns')

In [7]:
X

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,1,0


## First approach: 
### Reduce sample of banking customers without personal loan and complete machine learnign model

In [8]:
file_0=file[file['Personal Loan']==0]
file_1=file[file['Personal Loan']==1]

In [9]:
len(file_0)

4520

In [10]:
len(file_1)

480

#### For the over sample approach, I will increase the personal loan frequency by creating duplicates of banking customers from the personal loan yes dataframe

In [18]:
under_sample1=file_1.sample(4520, replace=True)

In [19]:
under_sample1

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
2246,2247,35,11,190,92093,3,3.1,2,266,1,0,0,0,0
4532,4533,48,22,133,90073,2,3.1,2,0,1,0,0,1,0
1373,1374,60,35,135,92612,3,0.3,3,0,1,0,0,1,0
1237,1238,38,13,169,92093,1,6.8,3,0,1,0,1,1,1
482,483,56,32,173,94022,1,4.6,2,88,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2968,2969,44,18,162,90840,4,1.3,1,301,1,0,0,0,0
4812,4813,29,4,184,92126,4,2.2,3,612,1,0,0,1,0
1405,1406,46,22,183,91605,1,3.1,2,0,1,0,1,1,1
941,942,38,13,129,92093,4,4.4,1,140,1,0,0,0,0


In [65]:
file_0

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


Next, I will concatenate these two dataframes and apply machine learning modelling

In [33]:
file=pd.concat([file_0, under_sample1], axis=0)

In [66]:
file

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2968,2969,44,18,162,90840,4,1.3,1,301,1,0,0,0,0
4812,4813,29,4,184,92126,4,2.2,3,612,1,0,0,1,0
1405,1406,46,22,183,91605,1,3.1,2,0,1,0,1,1,1
941,942,38,13,129,92093,4,4.4,1,140,1,0,0,0,0


In [67]:
X=file.drop('Personal Loan', axis='columns')
y=file['Personal Loan']

In [68]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

!pip install category_encoders
%matplotlib inline

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

In [70]:

print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

Train (7232, 13) (7232,)
Test (1808, 13) (1808,)


In [71]:
print(X_train)

        ID  Age  Experience  Income  ZIP Code  Family  CCAvg  Education  \
2653  2654   30           5     121     94555       2    3.1          1   
1998  1999   56          32     103     94111       3    4.0          3   
4501  4502   59          33      38     94132       3    2.2          3   
427    428   32           7      35     92521       3    1.3          1   
4311  4312   32           8      14     90034       3    0.9          3   
...    ...  ...         ...     ...       ...     ...    ...        ...   
1226  1227   60          36      14     90089       2    0.3          1   
2800  2801   52          26      28     93907       2    0.7          2   
1666  1667   51          25     190     95138       2    4.2          2   
2871  2872   65          39      82     94131       4    2.4          3   
1137  1138   51          26     134     90230       4    4.5          3   

      Mortgage  Securities Account  CD Account  Online  CreditCard  
2653       408                

In [72]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [73]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [75]:


model = GradientBoostingClassifier(learning_rate=0.1, max_depth=3, n_estimators=1000, subsample=0.5)

In [76]:
model.fit(X_train, y_train)

GradientBoostingClassifier(n_estimators=1000, subsample=0.5)

In [83]:
X_train

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
2653,2654,30,5,121,94555,2,3.1,1,408,0,0,1,0
1998,1999,56,32,103,94111,3,4.0,3,0,0,0,0,0
4501,4502,59,33,38,94132,3,2.2,3,178,1,1,1,1
427,428,32,7,35,92521,3,1.3,1,116,0,0,0,1
4311,4312,32,8,14,90034,3,0.9,3,111,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,1227,60,36,14,90089,2,0.3,1,109,0,0,1,1
2800,2801,52,26,28,93907,2,0.7,2,90,0,0,1,1
1666,1667,51,25,190,95138,2,4.2,2,0,0,0,1,0
2871,2872,65,39,82,94131,4,2.4,3,252,0,0,1,0


In [77]:
predictions=model.predict(X_test)


In [78]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1808 entries, 1592 to 3864
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  1808 non-null   int64  
 1   Age                 1808 non-null   int64  
 2   Experience          1808 non-null   int64  
 3   Income              1808 non-null   int64  
 4   ZIP Code            1808 non-null   int64  
 5   Family              1808 non-null   int64  
 6   CCAvg               1808 non-null   float64
 7   Education           1808 non-null   int64  
 8   Mortgage            1808 non-null   int64  
 9   Securities Account  1808 non-null   int64  
 10  CD Account          1808 non-null   int64  
 11  Online              1808 non-null   int64  
 12  CreditCard          1808 non-null   int64  
dtypes: float64(1), int64(12)
memory usage: 197.8 KB


In [79]:
classification_report(y_test,predictions)

'              precision    recall  f1-score   support\n\n           0       1.00      0.99      0.99       935\n           1       0.99      1.00      0.99       873\n\n    accuracy                           0.99      1808\n   macro avg       0.99      0.99      0.99      1808\nweighted avg       0.99      0.99      0.99      1808\n'

In [84]:

confusion_matrix(y_test,predictions)

array([[922,  13],
       [  0, 873]])

In [85]:
accuracy_score(y_test,predictions)

0.9928097345132744

In [86]:
print('The Precision Score- What proportion of positive identifications was actually correct?: ' + str(100*precision_score(y_test,predictions))+'%')
print('The Recall Score - What proportion of actual positives was identified correctly?: ' + str(100*recall_score(y_test,predictions))+'%')

The Precision Score- What proportion of positive identifications was actually correct?: 98.53273137697516%
The Recall Score - What proportion of actual positives was identified correctly?: 100.0%


In [87]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=15)
print('Cross-Validation Accuracy Scores', scores.mean())
recall = cross_val_score(model, X_train, y_train, cv=15, scoring='recall')
print('Recall', recall.mean())
precision = cross_val_score(model, X_train, y_train, cv=15, scoring='precision')
print('Precision', precision.mean())

Cross-Validation Accuracy Scores 0.9955748563181361
Recall 1.0
Precision 0.9908033015738985


In [ ]:
# example of grid searching key hyperparameters for GradientBoostingClassifier
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

# define models and parameters
model = GradientBoostingClassifier()
n_estimators = [10, 100, 1000]
learning_rate = [0.001, 0.01, 0.1]
subsample = [0.5, 0.7, 1.0]
max_depth = [3, 7, 9]
# define grid search
grid = dict(learning_rate=learning_rate, n_estimators=n_estimators, subsample=subsample, max_depth=max_depth)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='recall',error_score=0)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Accuracy: 98.45%
# Precision 94.33%
# Recall: 90.48%
